CALCULA OS PESOS DO PORTFÓLIO NOS INDICES

In [242]:
peso_single_names = 1
peso_indices = 1 - peso_single_names
custo_rebalanceamento = 0.0003
custo_ajuste = 0.0005


In [243]:
import pandas as pd

# Leitura dos arquivos Excel
df_port_comp = pd.read_excel('port_comp.xlsx', index_col=0)
df_peso_ibov = pd.read_excel('peso_ibov.xlsx', index_col=0)
df_peso_smal = pd.read_excel('peso_smal.xlsx', header=0)

# Remove a primeira coluna (Unnamed: 0) que só tem NaN
if 'Unnamed: 0' in df_peso_smal.columns:
    df_peso_smal = df_peso_smal.drop('Unnamed: 0', axis=1)

# Define a coluna 'Data' como índice e converte para datetime64
df_peso_smal['Data'] = pd.to_datetime(df_peso_smal['Data'])
df_peso_smal.set_index('Data', inplace=True)
# agora o índice é DatetimeIndex, sem .date

# Preenche valores NaN com zeros
df_port_comp.fillna(0, inplace=True)
df_peso_ibov.fillna(0, inplace=True)
df_peso_smal.fillna(0, inplace=True)

# Alinha (reindexa) e já preenche eventuais datas faltantes
df_peso_ibov = df_peso_ibov.reindex(df_port_comp.index).fillna(0)
df_peso_smal = df_peso_smal.reindex(df_port_comp.index).fillna(0)

# Multiplica elemento a elemento
df_peso_port_ibov = df_port_comp * df_peso_ibov
df_peso_port_smal = df_port_comp * df_peso_smal

# Soma totais
df_peso_port_ibov['Total_Ibov_%'] = df_peso_port_ibov.sum(axis=1)
df_peso_port_smal['Total_Smal_%'] = df_peso_port_smal.sum(axis=1)

print(df_peso_port_ibov.shape)        # (n_linhas, n_colunas)
print(df_peso_port_ibov.index.nunique())


print("IBOV:")
print(df_peso_port_ibov)
print("\nSMAL:")
print(df_peso_port_smal)

(1456, 172)
1456
IBOV:
                     TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  \
Data                                                                          
2019-09-02 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2019-09-03 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2019-09-04 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2019-09-05 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2019-09-06 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
2025-03-25 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2025-03-26 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2025-03-27 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2025-03-28 00:00:00      0      0    0.0    0.0    0.0    0.0      0    0.0   
2025-03-31 00:00:00      0   

Long Indices IBOV e SMAL

In [244]:
import pandas as pd

# ---------------------------
# Verificações iniciais
# ---------------------------

# Checa se os DataFrames necessários existem
if 'df_port_comp' not in globals():
    raise Exception("A variável 'df_port_comp' não foi definida.")
if 'df_peso_port_ibov' not in globals():
    raise Exception("A variável 'df_peso_port_ibov' não foi definida.")
if 'df_peso_port_smal' not in globals():
    raise Exception("A variável 'df_peso_port_smal' não foi definida.")

# Converte os índices para datetime se necessário (para correto alinhamento)
if not pd.api.types.is_datetime64_any_dtype(df_port_comp.index):
    df_port_comp.index = pd.to_datetime(df_port_comp.index)
if not pd.api.types.is_datetime64_any_dtype(df_peso_port_ibov.index):
    df_peso_port_ibov.index = pd.to_datetime(df_peso_port_ibov.index)
if not pd.api.types.is_datetime64_any_dtype(df_peso_port_smal.index):
    df_peso_port_smal.index = pd.to_datetime(df_peso_port_smal.index)

# ---------------------------
# Cálculos para IBOV
# ---------------------------

# Verifica se a coluna total para IBOV está presente
if 'Total_Ibov_%' not in df_peso_port_ibov.columns:
    raise Exception("A coluna 'Total_Ibov_%' não foi encontrada em df_peso_port_ibov.")

# Define as datas comuns entre df_port_comp e df_peso_port_ibov
common_dates_ibov = df_port_comp.index.intersection(df_peso_port_ibov.index)
if common_dates_ibov.empty:
    raise Exception("Não foram encontradas datas comuns para IBOV.")

# Define as colunas dos papéis: interseção entre as colunas de df_port_comp e de df_peso_port_ibov (excluindo 'Total_Ibov_%')
paper_cols_ibov = df_peso_port_ibov.columns.difference(['Total_Ibov_%'])
common_cols_ibov = df_port_comp.columns.intersection(paper_cols_ibov)
if common_cols_ibov.empty:
    raise Exception("Não foram encontradas colunas comuns (papéis) para IBOV.")

# Seleciona os DataFrames para os papéis e datas comuns (fazendo cópia para segurança)
df_port_comp_ibov = df_port_comp.loc[common_dates_ibov, common_cols_ibov].copy()
df_peso_ibov = df_peso_port_ibov.loc[common_dates_ibov, common_cols_ibov].copy()

# Converte os dados para numérico, caso haja formatações (usando errors='coerce' para transformar problemas em NaN)
df_port_comp_ibov = df_port_comp_ibov.apply(pd.to_numeric, errors='coerce')
df_peso_ibov = df_peso_ibov.apply(pd.to_numeric, errors='coerce')
total_ibov = pd.to_numeric(df_peso_port_ibov.loc[common_dates_ibov, 'Total_Ibov_%'], errors='coerce')

# Calcula 'n' para cada data como a soma dos valores do df_port_comp_ibov 
n_series_ibov = df_port_comp_ibov.sum(axis=1).replace(0, pd.NA)

# Aplica a fórmula para IBOV:
# ((1 - Total_Ibov_%)/n + Peso Port IBOVx) * Port Comp
componente_comum_ibov = (1 - total_ibov).div(n_series_ibov, axis=0)
df_long_indices_ibov = (df_peso_ibov.add(componente_comum_ibov, axis=0)) * df_port_comp_ibov

# Adiciona coluna de verificação (soma das colunas por data)
df_long_indices_ibov = df_long_indices_ibov.assign(Soma_verificacao = df_long_indices_ibov.sum(axis=1))

# ---------------------------
# Cálculos para SMAL
# ---------------------------

# Verifica se a coluna total para SMAL está presente
if 'Total_Smal_%' not in df_peso_port_smal.columns:
    raise Exception("A coluna 'Total_Smal_%' não foi encontrada em df_peso_port_smal.")

# Define as datas comuns entre df_port_comp e df_peso_port_smal
common_dates_smal = df_port_comp.index.intersection(df_peso_port_smal.index)
if common_dates_smal.empty:
    raise Exception("Não foram encontradas datas comuns para SMAL.")

# Define as colunas dos papéis para SMAL (excluindo 'Total_Smal_%')
paper_cols_smal = df_peso_port_smal.columns.difference(['Total_Smal_%'])
common_cols_smal = df_port_comp.columns.intersection(paper_cols_smal)
if common_cols_smal.empty:
    raise Exception("Não foram encontradas colunas comuns (papéis) para SMAL.")

# Seleciona os DataFrames para os papéis e datas comuns para SMAL
df_port_comp_smal = df_port_comp.loc[common_dates_smal, common_cols_smal].copy()
df_peso_smal = df_peso_port_smal.loc[common_dates_smal, common_cols_smal].copy()

# Converte os dados para numérico
df_port_comp_smal = df_port_comp_smal.apply(pd.to_numeric, errors='coerce')
df_peso_smal = df_peso_smal.apply(pd.to_numeric, errors='coerce')
total_smal = pd.to_numeric(df_peso_port_smal.loc[common_dates_smal, 'Total_Smal_%'], errors='coerce')

# Calcula 'n' para SMAL como a soma dos valores dos papéis por data
n_series_smal = df_port_comp_smal.sum(axis=1).replace(0, pd.NA)

# Aplica a fórmula para SMAL:
# ((1 - Total_Smal_%)/n + Peso Port SMALx) * Port Comp
componente_comum_smal = (1 - total_smal).div(n_series_smal, axis=0)
df_long_indices_smal = (df_peso_smal.add(componente_comum_smal, axis=0)) * df_port_comp_smal

# Adiciona coluna de verificação (soma dos valores por data)
df_long_indices_smal = df_long_indices_smal.assign(Soma_verificacao = df_long_indices_smal.sum(axis=1))

# ---------------------------
# Exibe os resultados
# ---------------------------

print("Resultado do df_long_indices_ibov:")
print(df_long_indices_ibov)

print("\nResultado do df_long_indices_smal:")
print(df_long_indices_smal)


Resultado do df_long_indices_ibov:
               TGMA3     ARML3  VAMO3  ALPA4  AZZA3  ASAI3     CEAB3  CRFB3  \
Data                                                                          
2019-09-02  0.020947  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-03  0.020953  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-04  0.020938  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-05  0.020865  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-06  0.020659  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
...              ...       ...    ...    ...    ...    ...       ...    ...   
2025-03-25  0.022877  0.022877    0.0    0.0    0.0    0.0  0.022877    0.0   
2025-03-26  0.022851  0.022851    0.0    0.0    0.0    0.0  0.022851    0.0   
2025-03-27  0.022849  0.022849    0.0    0.0    0.0    0.0  0.022849    0.0   
2025-03-28  0.022837  0.022837    0.0    0.0    0.0    0.0  0.022837    0.0   
2025-03-31  0.022

Peso Combinado

In [245]:
import pandas as pd

# --- Definição da lista de tickers (ajuste conforme necessário) ---
tickers = [
    'TGMA3', 'ARML3', 'VAMO3', 'ALPA4', 'AZZA3', 'ASAI3', 'CEAB3', 'CRFB3', 
    'ESPA3', 'GGPS3', 'GMAT3', 'GRND3', 'GUAR3', 'LJQQ3', 'LREN3', 'MGLU3', 
    'NTCO3', 'PETZ3', 'PNVL3', 'RADL3', 'SBFG3', 'SMFT3', 'VIVA3', 'VULC3', 
    'DXCO3', 'FRAS3', 'INTB3', 'LEVE3', 'MILS3', 'MYPK3', 'POMO4', 'PRNR3', 
    'PTBL3', 'RAPT4', 'TUPY3', 'WEGE3', 'CBAV3', 'CMIN3', 'CSNA3', 'GGBR4', 
    'KLBN11', 'RANI3', 'SUZB3', 'USIM5', 'VALE3', 'ALOS3', 'CURY3', 'CYRE3', 
    'DIRR3', 'EZTC3', 'IGTI11', 'JHSF3', 'MRVE3', 'MULT3', 'PLPL3', 'TEND3', 
    'BMOB3', 'LWSA3', 'NGRD3', 'POSI3', 'TOTS3', 'VLID3', 'HYPE3', 'BRIT3', 
    'DESK3', 'FIQE3', 'TIMS3', 'VIVT3', 'MOVI3', 'RENT3', 'RAIZ4', 'UGPA3', 
    'VBBR3', 'ENAT3', 'OPCT3', 'PETR4', 'PRIO3', 'RECV3', 'BRAV3', 'CCRO3', 
    'ECOR3', 'HBSA3', 'PORT3', 'RAIL3', 'STBP3', 'AESB3', 'AURE3', 'CMIG4', 
    'CPFE3', 'CPLE6', 'CSMG3', 'EGIE3', 'ELET6', 'ENEV3', 'ENGI11', 'EQTL3', 
    'NEOE3', 'SAPR11', 'SBSP3', 'SRNA3', 'ISAE4', 'ABCB4', 'B3SA3', 'BBAS3', 
    'BBDC4', 'BBSE3', 'BPAC11', 'BRBI11', 'CSUD3', 'CXSE3', 'IRBR3', 'ITUB4', 
    'QUAL3', 'PSSA3', 'SANB11', 'WIZC3', 'ABEV3', 'ZAMP3', 'BEEF3', 'BRFS3', 
    'CAML3', 'JBSS3', 'MDIA3', 'MRFG3', 'JALL3', 'KEPL3', 'SLCE3', 'SMTO3', 
    'SOJA3', 'TTEN3', 'ANIM3', 'COGN3', 'YDUQ3', 'FLRY3', 'HAPV3', 'ODPV3', 
    'ONCO3', 'RDOR3', 'XPBR31', 'MLAS3', 'CIEL3', 'SOMA3', 'BPAN4', 'BRSR6', 
    'LOGG3', 'EVEN3', 'MDNE3', 'LAVV3', 'DASA3', 'MATD3', 'VVEO3', 'CLSA3', 
    'CASH3', 'TRIS3', 'BLAU3', 'GFSA3', 'MTRE3', 'IFCM3', 'MBLY3', 'AMAR3', 
    'BRML3', 'TAEE11', 'ALUP11', 'AZUL4', 'UNIP6', 'ORVR3', 'AMBP3', 'CVCB3', 
    'TASA4', 'GOLL4', 'EMBR3'
]

# --- Alinhamento de datas ---
# Define as datas comuns entre df_peso_port_ibov e df_peso_port_smal
common_dates = df_peso_port_ibov.index.intersection(df_peso_port_smal.index)
if common_dates.empty:
    raise Exception("Não foram encontradas datas comuns entre df_peso_port_ibov e df_peso_port_smal.")

# --- Subconjunto dos tickers para cada DataFrame ---
# Seleciona as colunas dos tickers dos dois DataFrames para as datas comuns
df_ibov_tickers = df_peso_port_ibov.loc[common_dates, tickers].copy()
df_smal_tickers = df_peso_port_smal.loc[common_dates, tickers].copy()

# Converte para numérico, se necessário
df_ibov_tickers = df_ibov_tickers.apply(pd.to_numeric, errors='coerce')
df_smal_tickers = df_smal_tickers.apply(pd.to_numeric, errors='coerce')

# --- Cálculo do df_peso_combinado ---
# Soma os valores dos tickers de SMAL e de IBOV
df_peso_combinado = df_ibov_tickers.add(df_smal_tickers, fill_value=0)

# Cria a coluna "combinado" com a soma, linha a linha, dos valores dos tickers
df_peso_combinado['combinado'] = df_peso_combinado[tickers].sum(axis=1)

# --- Criação das colunas "Peso IBOV" e "Peso SMAL" ---
# Converte os valores dos totais para numérico
total_ibov = pd.to_numeric(df_peso_port_ibov.loc[common_dates, 'Total_Ibov_%'], errors='coerce')
total_smal = pd.to_numeric(df_peso_port_smal.loc[common_dates, 'Total_Smal_%'], errors='coerce')

# Cria a coluna "Peso IBOV": razão entre Total_Ibov_% e a coluna "combinado"
df_peso_combinado['Peso IBOV'] = total_ibov.div(df_peso_combinado['combinado'])

# Cria a coluna "Peso SMAL": razão entre Total_Smal_% e a coluna "combinado"
df_peso_combinado['Peso SMAL'] = total_smal.div(df_peso_combinado['combinado'])

# --- Exibe o resultado ---
print("Resultado do DataFrame 'df_peso_combinado':")
print(df_peso_combinado.head())


Resultado do DataFrame 'df_peso_combinado':
               TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                           
2019-09-02  0.004461    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03  0.004410    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04  0.004467    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05  0.004347    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-06  0.004290    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

            GGPS3  ...  UNIP6  ORVR3  AMBP3  CVCB3  TASA4  GOLL4  EMBR3  \
Data               ...                                                    
2019-09-02    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05    0.0  .

C:\Users\MiguelMachado\AppData\Local\Temp\ipykernel_25420\210274023.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_peso_combinado['combinado'] = df_peso_combinado[tickers].sum(axis=1)
C:\Users\MiguelMachado\AppData\Local\Temp\ipykernel_25420\210274023.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_peso_combinado['Peso IBOV'] = total_ibov.div(df_peso_combinado['combinado'])
C:\Users\MiguelMachado\AppData\Local\Temp\ipykernel_25420\210274023.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usua

Short Passivo IBOV e SMAL

In [246]:
import pandas as pd

# --- Para IBOV ---
# Verifica se a coluna 'Total_Ibov_%' está presente em df_peso_port_ibov
if 'Total_Ibov_%' not in df_peso_port_ibov.columns:
    raise Exception("A coluna 'Total_Ibov_%' não foi encontrada em df_peso_port_ibov.")

# Converte os valores da coluna para numérico (caso haja problemas de formatação)
total_ibov = pd.to_numeric(df_peso_port_ibov['Total_Ibov_%'], errors='coerce')

# Calcula a alavancagem de short passivo para IBOV: 1 / (1 - Total_Ibov_%)
df_alvancagem_short_passivo_ibov = pd.DataFrame(1 / (1 - total_ibov), index=df_peso_port_ibov.index)
df_alvancagem_short_passivo_ibov.columns = ['alvancagem_short_passivo_ibov']

# Exibe o resultado para IBOV
print("Resultado do df_alvancagem_short_passivo_ibov:")
print(df_alvancagem_short_passivo_ibov.head())

# --- Para SMAL ---
# Verifica se a coluna 'Total_Smal_%' está presente em df_peso_port_smal
if 'Total_Smal_%' not in df_peso_port_smal.columns:
    raise Exception("A coluna 'Total_Smal_%' não foi encontrada em df_peso_port_smal.")

# Converte os valores da coluna para numérico
total_smal = pd.to_numeric(df_peso_port_smal['Total_Smal_%'], errors='coerce')

# Calcula a alavancagem de short passivo para SMAL: 1 / (1 - Total_Smal_%)
df_alvancagem_short_passivo_smal = pd.DataFrame(1 / (1 - total_smal), index=df_peso_port_smal.index)
df_alvancagem_short_passivo_smal.columns = ['alvancagem_short_passivo_smal']

# Exibe o resultado para SMAL
print("\nResultado do df_alvancagem_short_passivo_smal:")
print(df_alvancagem_short_passivo_smal.head())


Resultado do df_alvancagem_short_passivo_ibov:
            alvancagem_short_passivo_ibov
Data                                     
2019-09-02                       1.836136
2019-09-03                       1.835603
2019-09-04                       1.836934
2019-09-05                       1.843388
2019-09-06                       1.861694

Resultado do df_alvancagem_short_passivo_smal:
            alvancagem_short_passivo_smal
Data                                     
2019-09-02                       1.317908
2019-09-03                       1.319312
2019-09-04                       1.316022
2019-09-05                       1.313690
2019-09-06                       1.313551


In [247]:
import pandas as pd

# --- Verificações e cálculo para IBOV ---
if 'df_alvancagem_short_passivo_ibov' not in globals():
    raise Exception("O DataFrame 'df_alvancagem_short_passivo_ibov' não foi definido.")
if 'df_peso_combinado' not in globals():
    raise Exception("O DataFrame 'df_peso_combinado' não foi definido.")
if 'Peso IBOV' not in df_peso_combinado.columns:
    raise Exception("A coluna 'Peso IBOV' não foi encontrada em df_peso_combinado.")

# Multiplica a alavancagem de short passivo do IBOV pelo Peso IBOV
prod_short_ibov = df_alvancagem_short_passivo_ibov['alvancagem_short_passivo_ibov'] * df_peso_combinado['Peso IBOV']
df_prod_short_passivo_ibov = pd.DataFrame(prod_short_ibov, index=df_alvancagem_short_passivo_ibov.index)
df_prod_short_passivo_ibov.columns = ['prod_short_passivo_ibov']

# --- Verificações e cálculo para SMAL ---
if 'df_alvancagem_short_passivo_smal' not in globals():
    raise Exception("O DataFrame 'df_alvancagem_short_passivo_smal' não foi definido.")
if 'Peso SMAL' not in df_peso_combinado.columns:
    raise Exception("A coluna 'Peso SMAL' não foi encontrada em df_peso_combinado.")

# Multiplica a alavancagem de short passivo do SMAL pelo Peso SMAL
prod_short_smal = df_alvancagem_short_passivo_smal['alvancagem_short_passivo_smal'] * df_peso_combinado['Peso SMAL']
df_prod_short_passivo_smal = pd.DataFrame(prod_short_smal, index=df_alvancagem_short_passivo_smal.index)
df_prod_short_passivo_smal.columns = ['prod_short_passivo_smal']

# --- Exibe os resultados ---
print("Resultado do df_prod_short_passivo_ibov:")
print(df_prod_short_passivo_ibov.head())

print("\nResultado do df_prod_short_passivo_smal:")
print(df_prod_short_passivo_smal.head())


Resultado do df_prod_short_passivo_ibov:
            prod_short_passivo_ibov
Data                               
2019-09-02                 1.200310
2019-09-03                 1.198429
2019-09-04                 1.202926
2019-09-05                 1.211232
2019-09-06                 1.228254

Resultado do df_prod_short_passivo_smal:
            prod_short_passivo_smal
Data                               
2019-09-02                 0.456371
2019-09-03                 0.457959
2019-09-04                 0.454218
2019-09-05                 0.450506
2019-09-06                 0.446935


In [248]:
import pandas as pd

# ============================
# Para IBOV
# ============================
# Verifica se as variáveis necessárias estão definidas
if 'df_prod_short_passivo_ibov' not in globals():
    raise Exception("O DataFrame 'df_prod_short_passivo_ibov' não foi definido.")
if 'df_peso_port_ibov' not in globals():
    raise Exception("O DataFrame 'df_peso_port_ibov' não foi definido.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# Seleciona do df_peso_port_ibov apenas as colunas correspondentes aos tickers, para as datas (índices) de df_prod_short_passivo_ibov
df_peso_ibov_tickers = df_peso_port_ibov.loc[df_prod_short_passivo_ibov.index, tickers].apply(pd.to_numeric, errors='coerce')

# Realiza a multiplicação: multiplica, linha a linha, o valor da série 'prod_short_passivo_ibov'
# pelos valores correspondentes dos tickers
df_short_passivo_ibov = df_peso_ibov_tickers.mul(df_prod_short_passivo_ibov['prod_short_passivo_ibov'], axis=0)

# Exibe o resultado para IBOV
print("Resultado do df_short_passivo_ibov:")
print(df_short_passivo_ibov.head())


# ============================
# Para SMAL
# ============================
# Verifica se as variáveis necessárias estão definidas
if 'df_prod_short_passivo_smal' not in globals():
    raise Exception("O DataFrame 'df_prod_short_passivo_smal' não foi definido.")
if 'df_peso_port_smal' not in globals():
    raise Exception("O DataFrame 'df_peso_port_smal' não foi definido.")

# Seleciona do df_peso_port_smal apenas as colunas correspondentes aos tickers, para as datas (índices) de df_prod_short_passivo_smal
df_peso_smal_tickers = df_peso_port_smal.loc[df_prod_short_passivo_smal.index, tickers].apply(pd.to_numeric, errors='coerce')

# Multiplica, linha a linha, o valor da série 'prod_short_passivo_smal'
# pelos valores correspondentes dos tickers
df_short_passivo_smal = df_peso_smal_tickers.mul(df_prod_short_passivo_smal['prod_short_passivo_smal'], axis=0)

# Exibe o resultado para SMAL
print("\nResultado do df_short_passivo_smal:")
print(df_short_passivo_smal.head())


Resultado do df_short_passivo_ibov:
            TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                        
2019-09-02    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-06    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

            GGPS3  ...    TAEE11  ALUP11  AZUL4  UNIP6  ORVR3  AMBP3  CVCB3  \
Data               ...                                                        
2019-09-02    0.0  ...  0.004220     0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03    0.0  ...  0.004256     0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04    0.0  ...  0.004283     0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05    0.0  ...  0.004

In [249]:
import pandas as pd

# Verifica se o DataFrame df_short_passivo_ibov existe
if 'df_short_passivo_ibov' not in globals():
    raise Exception("O DataFrame 'df_short_passivo_ibov' não foi definido.")

# Calcula a soma das linhas (axis=1) do df_short_passivo_ibov
# e cria um novo DataFrame com esta soma
df_soma_short_passivo_ibov = pd.DataFrame(df_short_passivo_ibov.sum(axis=1), 
                                           columns=['soma_short_passivo_ibov'])

# Exibe os primeiros registros do DataFrame resultante
print("Resultado do df_soma_short_passivo_ibov:")
print(df_soma_short_passivo_ibov.head())


Resultado do df_soma_short_passivo_ibov:
            soma_short_passivo_ibov
Data                               
2019-09-02                 0.546595
2019-09-03                 0.545548
2019-09-04                 0.548071
2019-09-05                 0.554163
2019-09-06                 0.568503


Peso Port Hib Adj.

In [250]:
import pandas as pd

# --- Verificações iniciais ---
if 'df_port_comp' not in globals():
    raise Exception("O DataFrame 'df_port_comp' não foi definido.")
if 'df_short_passivo_ibov' not in globals():
    raise Exception("O DataFrame 'df_short_passivo_ibov' não foi definido.")
if 'df_short_passivo_smal' not in globals():
    raise Exception("O DataFrame 'df_short_passivo_smal' não foi definido.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# --- Seleção dos dados para as empresas (tickers) ---
# Filtra as colunas de df_port_comp para conter apenas os tickers
df_port_comp_tickers = df_port_comp[tickers].copy()

# Calcula n para cada data como a soma dos valores dos tickers
n_series = df_port_comp_tickers.sum(axis=1)

# Calcula o termo (100%/n), considerando 100% como 1
term_div = 1 / n_series

# --- Preparação dos fatores de ajuste ---
# Vamos garantir que os DataFrames de short passivo estejam restritos aos tickers.
# Supondo que df_short_passivo_ibov e df_short_passivo_smal tenham, ou possuam colunas compatíveis com os tickers,
# selecionamos apenas essas colunas:
df_short_passivo_ibov_sub = df_short_passivo_ibov[tickers].copy()
df_short_passivo_smal_sub = df_short_passivo_smal[tickers].copy()

# Agora, somamos os DataFrames de short passivo e adicionamos o termo 1/n (que será adicionado a todas as colunas de cada linha)
# Usamos .add com axis=0 para que o termo (uma Series) seja somado a cada coluna da linha correspondente.
adj_factor = df_short_passivo_ibov_sub.add(df_short_passivo_smal_sub, axis=0).add(term_div, axis=0)

# --- Cálculo final ---
# Multiplica o fator ajustado pelo df_port_comp somente para os tickers
df_peso_port_hib_adj = adj_factor * df_port_comp_tickers

# --- Exibe o resultado ---
print("Resultado do df_peso_port_hib_adj:")
print(df_peso_port_hib_adj.head())


Resultado do df_peso_port_hib_adj:
               TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                           
2019-09-02  0.040498    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03  0.040481    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04  0.040490    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05  0.040420    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-06  0.040379    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

            GGPS3  ...    TAEE11    ALUP11  AZUL4  UNIP6  ORVR3  AMBP3  CVCB3  \
Data               ...                                                          
2019-09-02    0.0  ...  0.054332  0.045856    0.0    0.0    0.0    0.0    0.0   
2019-09-03    0.0  ...  0.054556  0.045864    0.0    0.0    0.0    0.0    0.0   
2019-09-04    0.0  ...  0.054717  0.045924    0.0    0.0    0.0    0.0    0.0   

In [251]:
import pandas as pd

# Verifica se o DataFrame df_peso_port_hib_adj está definido
if 'df_peso_port_hib_adj' not in globals():
    raise Exception("O DataFrame 'df_peso_port_hib_adj' não foi definido.")

# Calcula a soma de cada linha do df_peso_port_hib_adj (soma dos valores dos tickers)
df_gross_long = pd.DataFrame(df_peso_port_hib_adj.sum(axis=1), columns=['gross_long'])

# Exibe as primeiras linhas do DataFrame resultante
print("Resultado do df_gross_long:")
print(df_gross_long.head())


Resultado do df_gross_long:
            gross_long
Data                  
2019-09-02    1.656682
2019-09-03    1.656388
2019-09-04    1.657144
2019-09-05    1.661738
2019-09-06    1.675189


Peso Port Hib Adj. Limitado 100

In [252]:
import pandas as pd
import numpy as np

# Verifica se os DataFrames necessários existem
if 'df_gross_long' not in globals():
    raise Exception("O DataFrame 'df_gross_long' não foi definido.")
if 'df_port_comp' not in globals():
    raise Exception("O DataFrame 'df_port_comp' não foi definido.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# Calcula n: soma de cada linha dos tickers no df_port_comp
df_port_comp_tickers = df_port_comp[tickers].copy()
n_series = df_port_comp_tickers.sum(axis=1)

# Calcula o fator: (df_gross_long - 1) / n_series
# df_gross_long possui uma coluna chamada 'gross_long'
fator = (df_gross_long['gross_long'] - 1) / n_series

# Garante que o valor final seja não negativo, utilizando np.maximum
valores_redutor = np.maximum(fator, 0)

# Cria o DataFrame df_redutor com os valores calculados
df_redutor = pd.DataFrame(valores_redutor, index=df_gross_long.index, columns=['redutor'])

# Exibe os primeiros resultados

print("Resultado do df_redutor:")
print(df_redutor.head())


Resultado do df_redutor:
             redutor
Data                
2019-09-02  0.025257
2019-09-03  0.025246
2019-09-04  0.025275
2019-09-05  0.025451
2019-09-06  0.025969


In [253]:
import pandas as pd

# Verifica se os DataFrames e a lista de tickers estão definidos
if 'df_peso_port_hib_adj' not in globals():
    raise Exception("O DataFrame 'df_peso_port_hib_adj' não foi definido.")
if 'df_redutor' not in globals():
    raise Exception("O DataFrame 'df_redutor' não foi definido.")
if 'df_port_comp' not in globals():
    raise Exception("O DataFrame 'df_port_comp' não foi definido.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# Filtra df_port_comp para considerar apenas as empresas (tickers)
df_port_comp_tickers = df_port_comp[tickers].copy()

# Subtrai, para cada linha (data), o valor contido em df_redutor (coluna "redutor")
# de cada valor do df_peso_port_hib_adj. Note que o broadcasting é feito pela correspondência de índices.
df_ajustado = df_peso_port_hib_adj.subtract(df_redutor['redutor'], axis=0)

# Multiplica o DataFrame ajustado pelo df_port_comp restrito às colunas de tickers
df_peso_port_hib_adj_limitado_100 = df_ajustado * df_port_comp_tickers

# Exibe as primeiras linhas do resultado
print("Resultado do df_peso_port_hib_adj_limitado_100:")
print(df_peso_port_hib_adj_limitado_100.head())


Resultado do df_peso_port_hib_adj_limitado_100:
               TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                           
2019-09-02  0.015241   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-03  0.015235   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-04  0.015216   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-05  0.014968   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-06  0.014410   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   

            GGPS3  ...    TAEE11    ALUP11  AZUL4  UNIP6  ORVR3  AMBP3  CVCB3  \
Data               ...                                                          
2019-09-02   -0.0  ...  0.029075  0.020599   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-03   -0.0  ...  0.029311  0.020619   -0.0   -0.0   -0.0   -0.0   -0.0   
2019-09-04   -0.0  ...  0.029442  0.020649   -0.0   -0.0   -0.0   -

In [254]:
import pandas as pd

# Verifica se o DataFrame df_peso_port_hib_adj_limitado_100 está definido
if 'df_peso_port_hib_adj_limitado_100' not in globals():
    raise Exception("O DataFrame 'df_peso_port_hib_adj_limitado_100' não foi definido.")

# Calcula a soma de cada linha (axis=1) e cria o DataFrame df_gross_long_novo
df_gross_long_novo = pd.DataFrame(df_peso_port_hib_adj_limitado_100.sum(axis=1), 
                                  columns=['gross_long_novo'])

# Exibe as primeiras linhas do DataFrame resultante
print("Resultado do df_gross_long_novo:")
print(df_gross_long_novo.head())


Resultado do df_gross_long_novo:
            gross_long_novo
Data                       
2019-09-02              1.0
2019-09-03              1.0
2019-09-04              1.0
2019-09-05              1.0
2019-09-06              1.0


Peso Long Single Names

In [255]:
import pandas as pd

# Verifica se os DataFrames e a lista de tickers estão definidos
if 'df_port_comp' not in globals():
    raise Exception("O DataFrame 'df_port_comp' não foi definido.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# Seleciona apenas as colunas dos tickers em df_port_comp
df_port_comp_tickers = df_port_comp[tickers].copy()

# Calcula n como a soma de cada linha (para os tickers)
n_series = df_port_comp_tickers.sum(axis=1)

# Aplica a fórmula 1/n * df_port_comp para as empresas (tickers)
df_peso_long_single_names = df_port_comp_tickers.div(n_series, axis=0)

# Exibe as primeiras linhas do DataFrame resultante
print("Resultado do df_peso_long_single_names:")
print(df_peso_long_single_names.head())


Resultado do df_peso_long_single_names:
               TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                           
2019-09-02  0.038462    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-03  0.038462    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-04  0.038462    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-05  0.038462    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2019-09-06  0.038462    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

            GGPS3  ...    TAEE11    ALUP11  AZUL4  UNIP6  ORVR3  AMBP3  CVCB3  \
Data               ...                                                          
2019-09-02    0.0  ...  0.038462  0.038462    0.0    0.0    0.0    0.0    0.0   
2019-09-03    0.0  ...  0.038462  0.038462    0.0    0.0    0.0    0.0    0.0   
2019-09-04    0.0  ...  0.038462  0.038462    0.0    0.0    0.0    0.0    0

Posição Short Single Names

In [256]:
import pandas as pd

# Verificações iniciais
if 'df_gross_long_novo' not in globals():
    raise Exception("O DataFrame 'df_gross_long_novo' não foi definido.")
if 'df_peso_combinado' not in globals():
    raise Exception("O DataFrame 'df_peso_combinado' não foi definido.")
if 'peso_indices' not in globals():
    raise Exception("A variável 'peso_indices' não foi definida.")
if 'Peso IBOV' not in df_peso_combinado.columns:
    raise Exception("A coluna 'Peso IBOV' não foi encontrada em df_peso_combinado.")
if 'Peso SMAL' not in df_peso_combinado.columns:
    raise Exception("A coluna 'Peso SMAL' não foi encontrada em df_peso_combinado.")

# Cálculo para IBOV
produto_ibov = df_gross_long_novo['gross_long_novo'] * df_peso_combinado['Peso IBOV']
df_peso_short_ibov = -produto_ibov * peso_indices
df_peso_short_ibov = pd.DataFrame(df_peso_short_ibov, columns=['peso_short_ibov'])

# Cálculo para SMAL
produto_smal = df_gross_long_novo['gross_long_novo'] * df_peso_combinado['Peso SMAL']
df_peso_short_smal = -produto_smal * peso_indices
df_peso_short_smal = pd.DataFrame(df_peso_short_smal, columns=['peso_short_smal'])

# Exibe os resultados
print("Resultado do df_peso_short_ibov:")
print(df_peso_short_ibov.head())

print("\nResultado do df_peso_short_smal:")
print(df_peso_short_smal.head())

# --- salvar a tabela em Excel ---------------------------------
caminho_arquivo_ibov = "posicoes_Ibov.xlsx"          # nome do arquivo
df_peso_short_ibov.to_excel(
    caminho_arquivo_ibov,           # destino
    sheet_name="PosicoesIbov",  # nome da aba
    index=True                 # mantém a coluna‑índice “Data”
)

print(f"Tabela salva em {caminho_arquivo_ibov}")

# --- salvar a tabela em Excel ---------------------------------
caminho_arquivo_smal = "posicoes_Smal.xlsx"          # nome do arquivo
df_peso_short_smal.to_excel(
    caminho_arquivo_smal,           # destino
    sheet_name="PosicoesSmal",  # nome da aba
    index=True                 # mantém a coluna‑índice “Data”
)

print(f"Tabela salva em {caminho_arquivo_smal}")

Resultado do df_peso_short_ibov:
            peso_short_ibov
Data                       
2019-09-02             -0.0
2019-09-03             -0.0
2019-09-04             -0.0
2019-09-05             -0.0
2019-09-06             -0.0

Resultado do df_peso_short_smal:
            peso_short_smal
Data                       
2019-09-02             -0.0
2019-09-03             -0.0
2019-09-04             -0.0
2019-09-05             -0.0
2019-09-06             -0.0
Tabela salva em posicoes_Ibov.xlsx
Tabela salva em posicoes_Smal.xlsx


In [257]:
import pandas as pd

# Importar o arquivo
df_bottom_comp = pd.read_excel('bottom_comp.xlsx', index_col=0)

# Assumindo que df_peso_short_small já está disponível no seu código
# Podemos copiar o índice diretamente
df_bottom_comp.index = df_peso_short_smal.index.copy()

# Verificar o resultado
print("Dataframe com índice corrigido. Aqui estão as primeiras linhas:")
print(df_bottom_comp.head())


Dataframe com índice corrigido. Aqui estão as primeiras linhas:
            TGMA3  ARML3  VAMO3  ALPA4  AZZA3  ASAI3  CEAB3  CRFB3  ESPA3  \
Data                                                                        
2019-09-02      0      0      0      1      1      0      0      0      0   
2019-09-03      0      0      0      1      1      0      0      0      0   
2019-09-04      0      0      0      1      1      0      0      0      0   
2019-09-05      0      0      0      1      1      0      0      0      0   
2019-09-06      0      0      0      1      1      0      0      0      0   

            GGPS3  ...  TAEE11  ALUP11  AZUL4  UNIP6  ORVR3  AMBP3  CVCB3  \
Data               ...                                                      
2019-09-02      0  ...       0       0      0      1      0      0      0   
2019-09-03      0  ...       0       0      0      1      0      0      0   
2019-09-04      0  ...       0       0      0      1      0      0      0   
2019-09-05 

In [258]:
import pandas as pd

# Verifica se os DataFrames e variáveis necessários estão definidos
if 'df_port_comp' not in globals():
    raise Exception("O DataFrame 'df_port_comp' não foi definido.")
if 'df_bottom_comp' not in globals():
    raise Exception("O DataFrame 'df_bottom_comp' não foi definido.")
if 'peso_single_names' not in globals():
    raise Exception("A variável 'peso_single_names' não foi definida.")
if 'tickers' not in globals():
    raise Exception("A lista de tickers não foi definida.")

# Seleciona apenas as colunas dos tickers de df_port_comp para garantir a consistência
df_port_comp_tickers = df_port_comp[tickers].copy()

# Calcula n: a soma, para cada dia, dos valores dos tickers em df_port_comp
n_series = df_port_comp_tickers.sum(axis=1)

# Aplica a fórmula:
# (-100%/n * df_bottom_comp) * peso_single_names
# Sabendo que 100% equivale a 1, a expressão fica: (-1/n * df_bottom_comp) * peso_single_names
df_peso_short_single_names = df_bottom_comp.mul(-1 / n_series, axis=0) * peso_single_names

# Cria um DataFrame de verificação, que contém a soma das linhas de df_peso_short_single_names
df_verifica_peso_short_single_names = pd.DataFrame(df_peso_short_single_names.sum(axis=1),
                                                   columns=['soma_verificacao'])

# Exibe os resultados
print("Resultado do df_peso_short_single_names:")
print(df_peso_short_single_names.head())

print("\nResultado do df_verifica_peso_short_single_names:")
print(df_verifica_peso_short_single_names.head())


Resultado do df_peso_short_single_names:
            TGMA3  ARML3  VAMO3     ALPA4     AZZA3  ASAI3  CEAB3  CRFB3  \
Data                                                                       
2019-09-02   -0.0   -0.0   -0.0 -0.038462 -0.038462   -0.0   -0.0   -0.0   
2019-09-03   -0.0   -0.0   -0.0 -0.038462 -0.038462   -0.0   -0.0   -0.0   
2019-09-04   -0.0   -0.0   -0.0 -0.038462 -0.038462   -0.0   -0.0   -0.0   
2019-09-05   -0.0   -0.0   -0.0 -0.038462 -0.038462   -0.0   -0.0   -0.0   
2019-09-06   -0.0   -0.0   -0.0 -0.038462 -0.038462   -0.0   -0.0   -0.0   

            ESPA3  GGPS3  ...  TAEE11  ALUP11  AZUL4     UNIP6  ORVR3  AMBP3  \
Data                      ...                                                  
2019-09-02   -0.0   -0.0  ...    -0.0    -0.0   -0.0 -0.038462   -0.0   -0.0   
2019-09-03   -0.0   -0.0  ...    -0.0    -0.0   -0.0 -0.038462   -0.0   -0.0   
2019-09-04   -0.0   -0.0  ...    -0.0    -0.0   -0.0 -0.038462   -0.0   -0.0   
2019-09-05   -0.0   -0.0  

Peso Long Final

In [259]:
print(df_long_indices_ibov)
print(df_peso_short_ibov)
print(df_peso_short_smal)
print(df_long_indices_smal)
print(df_peso_long_single_names)
print(peso_single_names)

               TGMA3     ARML3  VAMO3  ALPA4  AZZA3  ASAI3     CEAB3  CRFB3  \
Data                                                                          
2019-09-02  0.020947  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-03  0.020953  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-04  0.020938  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-05  0.020865  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-06  0.020659  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
...              ...       ...    ...    ...    ...    ...       ...    ...   
2025-03-25  0.022877  0.022877    0.0    0.0    0.0    0.0  0.022877    0.0   
2025-03-26  0.022851  0.022851    0.0    0.0    0.0    0.0  0.022851    0.0   
2025-03-27  0.022849  0.022849    0.0    0.0    0.0    0.0  0.022849    0.0   
2025-03-28  0.022837  0.022837    0.0    0.0    0.0    0.0  0.022837    0.0   
2025-03-31  0.022821  0.022821    0.0    0.0    0.0 

In [260]:
import pandas as pd
import numpy as np

# Função para criar o df_peso_long_final
def criar_df_peso_long_final(df_peso_short_ibov, df_long_indices_ibov, 
                            df_peso_short_smal, df_long_indices_smal, 
                            df_peso_long_single_names, peso_single_names, tickers):
    """
    Cria df_peso_long_final usando a fórmula:
    = -df_peso_short_ibov * df_long_indices_ibov - df_peso_short_smal * df_long_indices_smal + peso_single_names * df_peso_long_single_names
    """
    # Inicializar dataframe final com as mesmas datas dos dataframes de entrada
    df_peso_long_final = pd.DataFrame(index=df_peso_short_ibov.index, columns=tickers)
    
    # Para cada ticker na lista de tickers comuns
    for ticker in tickers:
        # Aplicar a fórmula para este ticker
        # Multiplicação do peso short IBOV pelo long indices IBOV
        termo1 = -df_peso_short_ibov.values * df_long_indices_ibov[ticker].values
        
        # Multiplicação do peso short SMAL pelo long indices SMAL
        termo2 = -df_peso_short_smal.values * df_long_indices_smal[ticker].values
        
        # Multiplicação do peso_single_names pelo peso long single names
        termo3 = peso_single_names * df_peso_long_single_names[ticker].values
        
        # Somar os termos e atribuir ao ticker correspondente no dataframe final
        df_peso_long_final[ticker] = termo1.flatten() + termo2.flatten() + termo3
    
    # Adicionar coluna de soma para verificação
    df_peso_long_final['soma'] = df_peso_long_final[tickers].sum(axis=1)
    
    return df_peso_long_final

# Exemplo de uso do código
# Assumindo que você já tenha os dataframes carregados e a lista de tickers definida:

"""
# Exemplo:
# tickers = ['PETR4', 'VALE3', 'ITUB4', ...]  # Lista de tickers comuns

df_peso_long_final = criar_df_peso_long_final(
    df_peso_short_ibov,  # DataFrame com apenas uma coluna de valores para cada data
    df_long_indices_ibov,  # DataFrame com colunas para cada ticker
    df_peso_short_smal,  # DataFrame com apenas uma coluna de valores para cada data 
    df_long_indices_smal,  # DataFrame com colunas para cada ticker
    df_peso_long_single_names,  # DataFrame com colunas para cada ticker
    peso_single_names,  # Valor escalar
    tickers  # Lista de tickers comuns
)
"""

# Se você precisar de uma versão ainda mais vetorizada:
def criar_df_peso_long_final_v2(df_peso_short_ibov, df_long_indices_ibov, 
                              df_peso_short_smal, df_long_indices_smal, 
                              df_peso_long_single_names, peso_single_names, tickers):
    """
    Versão totalmente vetorizada da função
    """
    # Filtrar apenas os tickers necessários
    df_long_indices_ibov_filtered = df_long_indices_ibov[tickers]
    df_long_indices_smal_filtered = df_long_indices_smal[tickers]
    df_peso_long_single_names_filtered = df_peso_long_single_names[tickers]
    
    # Converter os pesos de short para um formato adequado para broadcast
    peso_short_ibov = df_peso_short_ibov.values.reshape(-1, 1)
    peso_short_smal = df_peso_short_smal.values.reshape(-1, 1)
    
    # Aplicar a fórmula de uma só vez
    df_peso_long_final = (
        -peso_short_ibov * df_long_indices_ibov_filtered
        -peso_short_smal * df_long_indices_smal_filtered
        +peso_single_names * df_peso_long_single_names_filtered
    )
    
    # Adicionar coluna de soma
    
    
    return df_peso_long_final

df_peso_long_final = criar_df_peso_long_final_v2(df_peso_short_ibov, df_long_indices_ibov, 
                              df_peso_short_smal, df_long_indices_smal, 
                              df_peso_long_single_names, peso_single_names, tickers)

print(df_peso_long_final)

               TGMA3     ARML3  VAMO3  ALPA4  AZZA3  ASAI3     CEAB3  CRFB3  \
Data                                                                          
2019-09-02  0.038462  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-03  0.038462  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-04  0.038462  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-05  0.038462  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
2019-09-06  0.038462  0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
...              ...       ...    ...    ...    ...    ...       ...    ...   
2025-03-25  0.028571  0.028571    0.0    0.0    0.0    0.0  0.028571    0.0   
2025-03-26  0.028571  0.028571    0.0    0.0    0.0    0.0  0.028571    0.0   
2025-03-27  0.028571  0.028571    0.0    0.0    0.0    0.0  0.028571    0.0   
2025-03-28  0.028571  0.028571    0.0    0.0    0.0    0.0  0.028571    0.0   
2025-03-31  0.028571  0.028571    0.0    0.0    0.0 

In [261]:
import pandas as pd

df_posicoes_net = (df_peso_long_final +df_peso_short_single_names )

print(df_posicoes_net)

# --- salvar a tabela em Excel ---------------------------------
caminho_arquivo_posicoes = "posicoes_net.xlsx"          # nome do arquivo
df_posicoes_net.to_excel(
    caminho_arquivo_posicoes,           # destino
    sheet_name="PosicoesNet",  # nome da aba
    index=True                 # mantém a coluna‑índice “Data”
)

print(f"Tabela salva em {caminho_arquivo_posicoes}")

               TGMA3     ARML3  VAMO3     ALPA4     AZZA3  ASAI3     CEAB3  \
Data                                                                         
2019-09-02  0.038462  0.000000    0.0 -0.038462 -0.038462    0.0  0.000000   
2019-09-03  0.038462  0.000000    0.0 -0.038462 -0.038462    0.0  0.000000   
2019-09-04  0.038462  0.000000    0.0 -0.038462 -0.038462    0.0  0.000000   
2019-09-05  0.038462  0.000000    0.0 -0.038462 -0.038462    0.0  0.000000   
2019-09-06  0.038462  0.000000    0.0 -0.038462 -0.038462    0.0  0.000000   
...              ...       ...    ...       ...       ...    ...       ...   
2025-03-25  0.028571  0.028571    0.0  0.000000 -0.028571    0.0  0.028571   
2025-03-26  0.028571  0.028571    0.0  0.000000 -0.028571    0.0  0.028571   
2025-03-27  0.028571  0.028571    0.0  0.000000 -0.028571    0.0  0.028571   
2025-03-28  0.028571  0.028571    0.0  0.000000 -0.028571    0.0  0.028571   
2025-03-31  0.028571  0.028571    0.0  0.000000 -0.028571    0.0